<a href="https://colab.research.google.com/github/R12942159/DeepLearning/blob/main/DLCV_hw1_timm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install timm

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch


# Get cuda from GPU device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

Using: cuda


In [4]:
# search file paths
from glob import glob


img_paths_train = glob('/content/drive/MyDrive/NTU_DLCV/p1_data/train_50/*.png') # *: all
img_paths_train = glob('/content/drive/MyDrive/NTU_DLCV/p1_data/train_50/*.png') # *: all
img_paths_val = glob('/content/drive/MyDrive/NTU_DLCV/p1_data/val_50/*.png')

In [5]:
# number of images
len(img_paths_train), len(img_paths_val)

(22500, 2500)

In [6]:
import torch
from PIL import Image
import numpy as np


class PretrainDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __len__(self):
        """number of samples"""
        return len(self.img_paths)

    def __getitem__(self, idx):
        """read 1 sample"""
        # Read img
        path = self.img_paths[idx] # get img path
        img = Image.open(path).convert('RGB') # read img
        # img_pil = Image.fromarray(img)
        # img = img_pil.resize((IMG_SIZE, IMG_SIZE), Image.LANCZOS)

        # transform img
        img = self.transform(img)

        # Read class index
        cls_idx = int((path.split('/')[-1]).split('_')[0])
        cls_idx = torch.tensor(cls_idx, dtype=torch.int64)

        return img, cls_idx

#### Build model and transform

In [7]:
import timm


model_name = 'beit_base_patch16_224.in22k_ft_in22k_in1k'
model = timm.create_model(model_name, pretrained=True, num_classes=50)

# create transform from pretrained model
data_cfg = timm.data.resolve_data_config(model.pretrained_cfg)
transform = timm.data.create_transform(**data_cfg)

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


#### Build Dataset

In [8]:
train_ds = PretrainDataset(img_paths_train, transform)
val_ds = PretrainDataset(img_paths_val, transform)

#### Build DataLoaders

In [9]:
BATCH_SIZE = 150
train_loader = torch.utils.data.DataLoader(train_ds, BATCH_SIZE, shuffle=True, pin_memory=True)
val_loader = torch.utils.data.DataLoader(val_ds, BATCH_SIZE, shuffle=False, pin_memory=True)

#### Train model

In [10]:
from tqdm.auto import tqdm # (optional) progress bar


def training(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.train() # to training mode. (The Dropout layer has a different behavior between the training mode and the evaluation mode.)
    epoch_loss, epoch_correct = 0, 0
    for batch_i, (x, y) in enumerate(tqdm(dataloader, leave=False)):
        x, y = x.to(device, non_blocking=True), y.to(device, non_blocking=True) # move data to GPU

        # zero the parameter gradients
        optimizer.zero_grad()

        # Compute prediction loss
        pred = model(x)
        loss = loss_fn(pred, y)

        # Optimization by gradients
        loss.backward() # backpropagation to compute gradients
        optimizer.step() # update model params

        # write to logs
        epoch_loss += loss.item() # tensor -> python value
        # (N, Class)
        epoch_correct += (pred.argmax(dim=1) == y).sum().item()

    # return avg loss of epoch, acc of epoch
    return epoch_loss/num_batches, epoch_correct/size


def testing(dataloader, model, loss_fn):
    size = len(dataloader.dataset) # number of samples
    num_batches = len(dataloader) # batches per epoch

    model.eval() # model to test mode.
    epoch_loss, epoch_correct = 0, 0

    # No gradient for test data
    with torch.no_grad():
        for batch_i, (x, y) in enumerate(dataloader):
            x, y = x.to(device), y.to(device)

            # Compute prediction loss
            pred = model(x)
            loss = loss_fn(pred, y)

            # write to logs
            epoch_loss += loss.item()
            epoch_correct += (pred.argmax(1) == y).sum().item()

    return epoch_loss/num_batches, epoch_correct/size

In [11]:
from torch import nn
import matplotlib.pyplot as plt
import os


best_acc = 0.7
cp_path = f'./p1_B_checkpoint_{model_name}'

def train(pretrained, freeze=False):
    print(f"pretrained weights: {pretrained}, freeze: {freeze}")
    # Model
    if pretrained:
        model = timm.create_model(model_name, pretrained=True, num_classes=50)
    else:
        model = timm.create_model(model_name, pretrained=False)
    if freeze:
        # Freeze model
        for param in model.parameters():
            param.requires_grad = False

    # model move to device
    model = model.to(device)

    loss_fn = nn.CrossEntropyLoss()
    # lower learning rate for finetuning
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    # schedule = ReduceLROnplateau(optimizer, 'min')

    EPOCHS = 30
    logs = {
        'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []
    }
    for epoch in tqdm(range(EPOCHS)):
        train_loss, train_acc = training(train_loader, model, loss_fn, optimizer)
        val_loss, val_acc = testing(val_loader, model, loss_fn)

        # # Note that step should be called after test_epoch()
        # schedule.step(val_loss)

        print(f'EPOCH: {epoch:04d} \
        train_loss: {train_loss:.4f}, train_acc: {train_acc:.3f} \
        val_loss: {val_loss:.4f}, val_acc: {val_acc:.3f} ')

        logs['train_loss'].append(train_loss)
        logs['train_acc'].append(train_acc)
        logs['val_loss'].append(val_loss)
        logs['val_acc'].append(val_acc)

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(optimizer.state_dict(), os.path.join(cp_path, 'best_optimizer.pth'))
            torch.save(model.state_dict(), os.path.join(cp_path, 'best_model.pth'))
            print("Model saved successfully!")


    # plot result
    plt.figure(figsize=(6, 3))
    plt.subplot(1, 2, 1)
    plt.title('Loss')
    plt.plot(logs['train_loss'])
    plt.plot(logs['val_loss'])
    plt.subplot(1, 2, 2)
    plt.title('Acc.')
    plt.plot(logs['train_acc'])
    plt.plot(logs['val_acc'])
    plt.show()

    # Save model
    # torch.save(model.state_dict(), "last.pth")

#### Transfer Learning

In [13]:
# train(pretrained=True, freeze=False)
train(pretrained=True, freeze=True)

pretrained weights: True, freeze: True


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/75 [00:00<?, ?it/s]

OutOfMemoryError: ignored